In [ ]:
%%javascript

require(['widgets/js/widget', 'widgets/js/manager', 'jquery', 'underscore'], function(widget, manager, $, _) {
    var GoogleMapsView = widget.DOMWidgetView.extend({
        initialize: function() {
            console.log(_);
            var mapsDeferred = this.mapsDeferred = $.Deferred();
            if (window.google && window.google.maps) {
                mapsDeferred.resolve();
            } else {
                window.googleMapsCallback = function() {
                    mapsDeferred.resolve();
                }
                require(
                    ['http://maps.googleapis.com/maps/api/js?callback=googleMapsCallback'], 
                    function() {}, 
                    function() {});
            }
        },
        
        render: function() {
            $.when(this.mapsDeferred.promise()).then(function() {
                this.$el.empty();
                _.defer(function() {
                    this.$map = $('<div style="height: 100px; width: 400px;"></div>');
                    this.$el.append(this.$map);
                    this.map = new google.maps.Map(this.$map.get(0), {
                        center: {lat: -34.397, lng: 150.644},
                        zoom: 8
                    });
                }.bind(this));
            }.bind(this));
        },
    });
    
    manager.WidgetManager.register_widget_view('GoogleMapsView', GoogleMapsView);
});

In [ ]:
from ipywidgets import widgets
from traitlets import Unicode, CInt

class GoogleMapsView(widgets.DOMWidget):
    _view_name = Unicode('GoogleMapsView', sync=True)
    value = CInt(0, sync=True)

In [ ]:
w = GoogleMapsView()

In [ ]:
w

In [ ]:
z = GoogleMapsView()

In [ ]:
z

In [ ]:
z